<a href="https://colab.research.google.com/github/bickrombishsass/Train-Time-Table/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import sys
import datetime

In [ ]:
source_stn = input("Enter Source Station code : ") 
dest_stn = input("Enter Destination Station code : ") 
date, month, year = input("Enter date : ").split('/') 

Enter Source Station code : NZM
Enter Destination Station code : HWH
Enter date : 20/05/20


In [ ]:
url = 'https://www.railyatri.in/booking/trains-between-stations?from_code='+\
	  source_stn+'&from_name=SRC+&journey_date='+date+'%2F'+month+'%2F'+year+'%2F'+'&to_code='+\
	  dest_stn+'&to_name=DST+&user_id=1&user_token=6'

In [ ]:
url

'https://www.railyatri.in/booking/trains-between-stations?from_code=NZM&from_name=SRC+&journey_date=20%2F05%2F20%2F&to_code=HWH&to_name=DST+&user_id=1&user_token=6'

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
trains = []

In [ ]:
for row in soup.select('tr.tbs-main-row'):
	data = {}
	title = row.find('p', {'class': 'train-name-title'}).text.strip('\n')
	data['trn_no'] = title[:5]
	data['trn_name'] = title[8:]
	data['schedule'] = 'https://erail.in/train-enquiry/'+title[:5]
	spans = row.find_all('span')[:6]
	data['source'] = source_stn
	data['departure time'] = spans[1].text
	data['destination'] = dest_stn
	data['arrival time'] = spans[4].text
	data['duration'] = spans[5].text
	data['fares'] = []
	classes = row.find_all('div', {'class': 'coach'})
	for i in range(len(classes)//2):
		pair = {}
		deets = classes[i].find_all('p')
		pair['Class'] = deets[0].text
		pair['Price'] = deets[1].text
		if pair['Price'] == '₹ NA':
			continue
		data['fares'].append(pair)
	if len(data['fares']) != 0:
		trains.append(data)

In [ ]:
import pandas as pd
df=pd.DataFrame(trains)
df.shape

(0, 0)